### 1. 패키지 설치 (LCEL 적용 버전)

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-pinecone langchainhub

/Users/lhj/inflearn-langchain-rag/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


### 2. Knowledge Base 구성을 위한 데이터 생성
- split 된 데이터 chunk를 LLM에게 전달하면 토큰 절약 가능 -> 비용 감소와 답변 생성시간 감소의 효과
- LangChain에서 다양한 TextSplitter들을 제공
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
    - chunk_size 는 split 된 chunk의 최대 크기
    - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [7]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [8]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [9]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'index-2'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

#인덱스 새로 만들기 
database = PineconeVectorStore.from_documents(
    document_list, embedding, 
    index_name=index_name
)

### 3. 관련 정보를 가져오는 Retrieval (검색)
- Chroma에 저장한 데이터에서 질문 쿼리와 관련있는 데이터를 가져오는 과정
- similarity_search(): 유사도 검색 제공 
- `k` 값을 조절해서 얼마나 많은 연관 데이터를 불러올지 결정

### * LCEL 적용: 쿼리 변환 체인 생성
- 프롬프트 | LLM | OutputParser를 파이프라인으로 연결
- 한 번의 invoke()로 전체 프로세스 실행

In [10]:
# LCEL: 쿼리 변환 체인 생성
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")


dictionary = ["사람을 나타내는 표현 -> 거주자"]
query_transform_prompt = ChatPromptTemplate.from_template(
    """사용자의 질문을 보고, 키워드 사전을 참고해서 사용자의 질문을 변경해주세요. 
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다. 
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    사용자의 질문: {question}
    """
)
# 쿼리 변환 체인: 프롬프트 | LLM | 문자열 파서
query_transform_chain = (
    query_transform_prompt 
    | llm 
    | StrOutputParser()
)

### 4. Augmentation(증강)을 위한 Prompt 활용
- Retrieval된 데이터를 프롬프트를 활용하여 LLM에 전달 
- LangChain에서 제공하는 프롬포트("rlm/rag-prompt")를 사용해도 되고, 본인이 직접 작성해도 된다. 

### * LCEL 적용: Retriever 생성
- similarity_search 대신 as_retriever() 사용
- 체인에 통합 가능한 형태로 변환

In [11]:
# LCEL: Retriever 생성
retriever = database.as_retriever(
    search_kwargs={"k": 3}
)

### * LCEL 적용: RAG 체인 생성
- RunnablePassthrough를 사용해 context와 question을 병렬 처리
- 검색 → 포맷팅 → 프롬프트 → LLM을 하나의 체인으로 연결

In [12]:
# LCEL: Context 포맷팅 함수
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    """검색된 문서들을 하나의 context 문자열로 포맷팅"""
    return "\n\n---\n\n".join([doc.page_content for doc in docs])

# RAG 프롬프트 정의
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 최고의 한국 소득세 전문가입니다. 
                주어진 context를 기반으로 질문에 답변하세요."""
    ),
    ("human", """Context: {context}
                Question: {question}"""
    )
])

# LCEL: RAG 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs,  # 검색 후 포맷팅
        "question": RunnablePassthrough()     # 질문 그대로 전달
    }
    | rag_prompt                              # 프롬프트 생성
    | llm                                     # LLM 호출
    | StrOutputParser()                       # 문자열 추출
)

### 5. 답변 생성 (Generation)
- 프롬포트를 LLM에게 전달하여 답변 생성 

### * LCEL 적용: 전체 파이프라인 통합
- 쿼리 변환 체인 + RAG 체인을 하나로 연결
- 원본 질문 입력 → 변환 → 검색 → 답변 생성까지 한 번에 실행

In [13]:
# LCEL: 전체 파이프라인 통합
# 방법 1: 순차적으로 연결
full_chain = query_transform_chain | rag_chain

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# 전체 파이프라인 실행
answer = full_chain.invoke({
    "question": query, 
    "dictionary": dictionary
})

print("=" * 50)
print(f"질문: {query}")
print("=" * 50)
print(f"답변:\n{answer}")
print("=" * 50)

질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
답변:
연봉 5천만원인 거주자의 소득세를 계산해 보겠습니다.

1. **과세표준**: 5,000만원입니다.
2. **적용되는 세율**: 이 금액은 '1,400만원 초과 5,000만원 이하' 구간에 해당하므로 다음과 같은 세율을 적용합니다:
   - 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)

   5,000만원 - 1,400만원 = 3,600만원입니다.
   이 금액에 15%를 적용하면: 
   \( 3,600만원 \times 0.15 = 540만원 \)

3. **종합소득산출세액**: 
   \( 84만원 + 540만원 = 624만원 \)

따라서, 연봉 5천만원인 거주자의 소득세는 **624만원**입니다.


### * LCEL의 추가 기능: 스트리밍
- .stream() 메서드로 실시간 응답 확인
- 긴 답변을 기다리지 않고 즉시 출력 시작

In [14]:
# LCEL: 스트리밍 예제
print("스트리밍으로 답변 생성 중...\n")
for chunk in full_chain.stream({"question": query, "dictionary": dictionary}):
    print(chunk, end="", flush=True)
print("\n\n스트리밍 완료!")

스트리밍으로 답변 생성 중...

연봉 5천만원인 거주자의 소득세를 계산하기 위해, 종합소득 과세표준에 해당하는 5천만원에 대한 세율을 적용해야 합니다.

종합소득 과세표준이 5천만원인 경우, 세율은 다음과 같습니다:

1. 1,400만원 이하: 과세표준의 6% → 1,400만원 × 0.06 = 84만원
2. 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15% 
   = 84만원 + (3,600만원 × 0.15) 
   = 84만원 + 540만원 
   = 624만원

따라서, 연봉 5천만원인 거주자의 소득세는 **624만원**입니다.

스트리밍 완료!


### * LCEL의 추가 기능: 배치 처리
- .batch() 메서드로 여러 질문을 한 번에 처리
- 효율적인 병렬 처리

In [15]:
# LCEL: 배치 처리 예제
questions = [
    {"question": "연봉 5천만원인 직장인의 소득세는 얼마인가요?", "dictionary": dictionary},
    {"question": "연봉 3천만원인 사람의 소득세는 얼마인가요?", "dictionary": dictionary},
    {"question": "퇴직소득세는 어떻게 계산하나요?", "dictionary": dictionary}
]

# 여러 질문을 한 번에 처리
answers = full_chain.batch(questions)

# 결과 출력
for i, (q, a) in enumerate(zip(questions, answers), 1):
    print(f"\n질문 {i}: {q['question']}")
    print(f"답변 {i}: {a[:100]}...")  # 답변의 처음 100자만 출력
    print("-" * 50)


질문 1: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
답변 1: 연봉이 5천만원인 거주자의 소득세를 계산하기 위해 해당 금액의 종합소득과세표준을 알아야 합니다.

1. 연봉 5천만원은 소득세 계산 시 종합소득과세표준이 5,000만원에 해당합니다...
--------------------------------------------------

질문 2: 연봉 3천만원인 사람의 소득세는 얼마인가요?
답변 2: 연봉이 3천만원인 거주자의 종합소득세를 계산하는 과정은 다음과 같습니다.

1. **종합소득 과세표준**: 3천만원
2. **세율표**에 따라, 3천만원은 "1억5천만원 초과 3억...
--------------------------------------------------

질문 3: 퇴직소득세는 어떻게 계산하나요?
답변 3: 퇴직소득세는 다음의 절차에 따라 계산됩니다:

1. **퇴직소득의 정의**: 퇴직소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 구성됩니다.
   - 공적연금 관련법에 따라 ...
--------------------------------------------------


### 체인 생성  -> 통합 파이프라인 한 눈에 보기 

In [ ]:
# 1. 쿼리 변환 체인: 프롬프트 | LLM | 문자열 파서
dictionary = ["사람을 나타내는 표현 -> 거주자"]
query_transform_prompt = ChatPromptTemplate.from_template(
    """사용자의 질문을 보고, 키워드 사전을 참고해서 사용자의 질문을 변경해주세요. 
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다. 
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    사용자의 질문: {question}
    """
)
query_transform_chain = (
    query_transform_prompt  #dic_value = query_transform_prompt.invoke({"question": query, "dictionary": dictionary})
    | llm                   #ai_response = llm.invoke(dic_value)
    | StrOutputParser()     #new_query =ai_response.content
)

# 2. RAG 체인 생성
def format_docs(docs):
    """검색된 문서들을 하나의 context 문자열로 포맷팅"""
    return "\n\n---\n\n".join([doc.page_content for doc in docs])


# LCEL: Retriever 생성
retriever = database.as_retriever(
    search_kwargs={"k": 3}
)
#docs = retriever.invoke("연봉 5천만원인 거주자의 소득세는?")

# RAG 프롬프트 정의
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 최고의 한국 소득세 전문가입니다. 
                주어진 context를 기반으로 질문에 답변하세요."""
    ),
    ("human", """Context: {context}
                Question: {question}"""
    )
])

rag_chain = (
    {   # 결과: {"context": "포맷팅된 문서들", "question": "new_query"}
        "context": retriever | format_docs,        # ① docs = retriever.invoke(new_query) → ② format_docs(docs)
        "question": RunnablePassthrough()     # new_query 그대로 전달
    }   
    | rag_prompt                              # chat_value = rag_prompt.invoke({"context": context, "question": new_query})
    | llm                                     # ai_response = llm.invoke(chat_value)
    | StrOutputParser()                       # result = ai_response.content
)

# 3. 전체 파이프라인 통합 
full_chain = query_transform_chain | rag_chain

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

answer = full_chain.invoke({
    "question": query, 
    "dictionary": dictionary
})
